In [1]:
%uv add moccasin
%uv sync

Resolved 94 packages in 1ms
Audited 88 packages in 0.04ms
Note: you may need to restart the kernel to use updated packages.
Resolved 94 packages in 1ms
Audited 88 packages in 0.03ms
Note: you may need to restart the kernel to use updated packages.


In [1]:
from moccasin import setup_notebook

setup_notebook()

In [7]:
from moccasin.config import get_active_network, get_config

active_network = get_active_network()
print(active_network.name)

eth-forked


In [4]:
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple
from moccasin.config import get_active_network
import boa

STARTING_ETH_BALANCE = int(1000e18)
STARTING_USDC_BALANCE = int(100e6)
STARTING_WETH_BALANCE = int(1e18)

def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)

def _add_token_balance(usdc: ABIContract, weth: ABIContract):
    weth.deposit(value=STARTING_WETH_BALANCE)
    print("USDC balance before:", usdc.balanceOf(boa.env.eoa))
    our_address = boa.env.eoa
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    usdc.configureMinter(our_address, STARTING_USDC_BALANCE)
    usdc.mint(our_address, STARTING_USDC_BALANCE)
    print("USDC balance after:", usdc.balanceOf(our_address))

def setup_script() -> Tuple[ABIContract, ABIContract, ABIContract, ABIContract]:
    print("Starting setup script")

    active_network = get_active_network()
    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")
    if active_network.is_local_or_forked_network():
        # 1. Give ourselves some ETH
        # 2. Give ourselves some USDC and WETH
        _add_eth_balance()
        _add_token_balance(usdc, weth)


def moccasin_main():
    setup_script()

moccasin_main()

Starting setup script
USDC balance before: 0
USDC balance after: 100000000


In [8]:
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")

In [16]:
# To reload config
config = get_config()
config.reload()
active_network = config.get_active_network()
# We want to call getPool() to know where to send our money
aavev3_pool_address_provider = active_network.manifest_named("aavev3_pool_address_provider")
pool_address = aavev3_pool_address_provider.getPool()
print(pool_address)

0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [17]:
config.reload()
active_network = config.get_active_network()
pool_contract = active_network.manifest_named("pool", address=pool_address)

In [20]:
REFERRAL_CODE = 0

def deposit(pool_contract: ABIContract, token: ABIContract, amount: int):
    allowed_amount = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amount < amount:
        token.approve(pool_contract.address, amount)
    print(f"Depositing {amount} {token.name()} into Aave contract {pool_contract.address}")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFERRAL_CODE)

usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)

if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)

if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)

(
    total_collateral_base,
    total_debt_base,
    available_borrows_base,
    current_liquidation_threshold,
    ltv,
    health_factor
) = pool_contract.getUserAccountData(boa.env.eoa)

print(f"""User account data:
    Total collateral base: {total_collateral_base}
    Total debt base: {total_debt_base}
    Available borrows base: {available_borrows_base}
    Current liquidation threshold: {current_liquidation_threshold}
    LTV: {ltv}
    Health factor: {health_factor}
""")

Depositing 100000000 USD Coin into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
Depositing 1000000000000000000 Wrapped Ether into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
User account data:
    Total collateral base: 351659400000
    Total debt base: 0
    Available borrows base: 282523161960
    Current liquidation threshold: 8285
    LTV: 8034
    Health factor: 115792089237316195423570985008687907853269984665640564039457584007913129639935



In [23]:
# 30% USDC and 70% WETH
config.reload()
active_network = config.get_active_network()

aave_protocol_data_provider = active_network.manifest_named("aave_protocol_data_provider")
a_tokens = aave_protocol_data_provider.getAllATokens()
print(a_tokens)

[('aEthWETH', Address('0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8')), ('aEthwstETH', Address('0x0B925eD163218f6662a35e0f0371Ac234f9E9371')), ('aEthWBTC', Address('0x5Ee5bf7ae06D1Be5997A1A72006FE6C607eC6DE8')), ('aEthUSDC', Address('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c')), ('aEthDAI', Address('0x018008bfb33d285247A21d44E50697654f754e63')), ('aEthLINK', Address('0x5E8C8A7243651DB1384C0dDfDbE39761E8e7E51a')), ('aEthAAVE', Address('0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9')), ('aEthcbETH', Address('0x977b6fc5dE62598B08C85AC8Cf2b745874E8b78c')), ('aEthUSDT', Address('0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a')), ('aEthrETH', Address('0xCc9EE9483f662091a1de4795249E24aC0aC2630f')), ('aEthLUSD', Address('0x3Fe6a295459FAe07DF8A0ceCC36F37160FE86AA9')), ('aEthCRV', Address('0x7B95Ec873268a6BFC6427e7a28e396Db9D0ebc65')), ('aEthMKR', Address('0x8A458A9dc9048e005d22849F470891b840296619')), ('aEthSNX', Address('0xC7B4c17861357B8ABB91F25581E7263E08DCB59c')), ('aEthBAL', Address('0x2516E7B3F76

In [27]:
for a_token in a_tokens:
    if "WETH" in a_token[0]:
        a_weth = active_network.manifest_named("weth", address=a_token[1])
    if "USDC" in a_token[0]:
        a_usdc = active_network.manifest_named("usdc", address=a_token[1])

In [28]:
print(a_weth)
print(a_usdc)

<weth interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>
<usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>


In [30]:
# get how much they are worth, to figure out what our portfolio makeup is
a_usdc_balance = a_usdc.balanceOf(boa.env.eoa) # 6 decimals
a_weth_balance = a_weth.balanceOf(boa.env.eoa) # 18 decimals

a_usdc_balance_normalized = a_usdc_balance / 10 ** 6
a_weth_balance_normalized = a_weth_balance / 10 ** 18

print(f"USDC balance normalized: {a_usdc_balance_normalized}")
print(f"WETH balance normalized: {a_weth_balance_normalized}")

USDC balance normalized: 100.0
WETH balance normalized: 1.0


In [39]:
# Chainlink pricefeeds are slightly delayed and out of sync with the actual price
config.reload()
active_network = config.get_active_network()

def get_price(feed_name: str) -> float:
    price_feed = active_network.manifest_named(feed_name)
    price =  price_feed.latestAnswer()
    decimals = price_feed.decimals()
    return price / 10 ** decimals

usdc_price = get_price("usdc_usd_price_feed")
eth_price = get_price("eth_usd_price_feed")

print(f"USDC price: {usdc_price}")
print(f"ETH price: {eth_price}")


USDC price: 1.00004
ETH price: 3416.59


In [48]:
usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * eth_price
total_value = usdc_value + weth_value

usdc_percent_allocation = usdc_value / total_value
weth_percent_allocation = weth_value / total_value

target_usdc_percent_allocation = 0.3    
target_weth_percent_allocation = 0.7

target_usdc_value = total_value * target_usdc_percent_allocation
target_weth_value = total_value * target_weth_percent_allocation

print(f"Total value: {total_value}")
# Show only 2 decimal places
print(f"USDC percent allocation: {usdc_percent_allocation:.2f} => Target: {target_usdc_percent_allocation}")
print(f"WETH percent allocation: {weth_percent_allocation:.2f} => Target: {target_weth_percent_allocation}")

print(f"USDC value: {usdc_value} => Target: {target_usdc_value}")
print(f"WETH value: {weth_value} => Target: {target_weth_value}")

BUFFER = 0.1

needs_rebalancing = (
    abs(usdc_value - target_usdc_value) > BUFFER or
    abs(weth_value - target_weth_value) > BUFFER
)

print(f"Needs rebalancing: {needs_rebalancing}")


Total value: 3516.594
USDC percent allocation: 0.03 => Target: 0.3
WETH percent allocation: 0.97 => Target: 0.7
USDC value: 100.004 => Target: 1054.9782
WETH value: 3416.59 => Target: 2461.6158
Needs rebalancing: True


In [65]:
usdc_data = {
    "balance": a_usdc_balance_normalized,
    "price": usdc_price,
    "contract": usdc
}

weth_data = {
    "balance": a_weth_balance_normalized,
    "price": eth_price,
    "contract": weth
}

target_allocations = {
    "usdc": 0.3,
    "weth": 0.7
}

def calculate_rebalancing_trades(
    usdc_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    weth_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    target_allocations: dict[str, float],  # {"usdc": 0.3, "weth": 0.7}
) -> dict[str, dict]:
    """
    Calculate the trades needed to rebalance a portfolio of USDC and WETH.

    Args:
        usdc_data: Dict containing USDC balance, price and contract
        weth_data: Dict containing WETH balance, price and contract
        target_allocations: Dict of token symbol to target allocation (must sum to 1)

    Returns:
        Dict of token symbol to dict containing contract and trade amount:
            {"usdc": {"contract": Contract, "trade": int},
             "weth": {"contract": Contract, "trade": int}}
    """
    # Calculate current values
    usdc_value = usdc_data["balance"] * usdc_data["price"]
    weth_value = weth_data["balance"] * weth_data["price"]
    total_value = usdc_value + weth_value

    # Calculate target values
    target_usdc_value = total_value * target_allocations["usdc"]
    target_weth_value = total_value * target_allocations["weth"]

    # Calculate trades needed in USD
    usdc_trade_usd = target_usdc_value - usdc_value
    weth_trade_usd = target_weth_value - weth_value

    # Convert to token amounts
    return {
        "usdc": {
            "contract": usdc_data["contract"],
            "trade": usdc_trade_usd / usdc_data["price"],
        },
        "weth": {
            "contract": weth_data["contract"],
            "trade": weth_trade_usd / weth_data["price"],
        },
    }


trades = calculate_rebalancing_trades(usdc_data, weth_data, target_allocations)
trades



{'usdc': {'contract': <usdc interface at 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48>,
  'trade': 954.9360025598976},
 'weth': {'contract': <weth interface at 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2>,
  'trade': -0.27951091585469723}}

In [66]:
def print_token_balances():
    print(f"USDC balance: {usdc.balanceOf(boa.env.eoa)}")
    print(f"WETH balance: {weth.balanceOf(boa.env.eoa)}")
    print(f"Aave USDC balance: {a_usdc.balanceOf(boa.env.eoa)}")
    print(f"Aave WETH balance: {a_weth.balanceOf(boa.env.eoa)}")

print_token_balances()

USDC balance: 0
WETH balance: 279510915854697216
Aave USDC balance: 100000000
Aave WETH balance: 720489084145302785


In [67]:
# if weth is negative, we need to sell weth
if trades["weth"]["trade"] < 0:
    weth_amount_to_sell_in_wei = int(trades["weth"]["trade"] * 10 ** 18)
    a_weth.approve(pool_contract.address, weth_amount_to_sell_in_wei)
    pool_contract.withdraw(weth.address, weth_amount_to_sell_in_wei, boa.env.eoa)

# if usdc is negative, we need to sell usdc
if trades["usdc"]["trade"] < 0:
    usdc_amount_to_sell_without_decimals = int(trades["usdc"]["trade"] * 10 ** 6)
    a_usdc.approve(pool_contract.address, usdc_amount_to_sell_without_decimals)
    pool_contract.withdraw(usdc.address, usdc_amount_to_sell_without_decimals, boa.env.eoa)

print_token_balances()

USDC balance: 0
WETH balance: 279510915854697216
Aave USDC balance: 100000000
Aave WETH balance: 720489084145302785


In [71]:
# struct ExactInputSingleParams {
#     address tokenIn;
#     address tokenOut;
#     uint24 fee;
#     address recipient;
#     uint256 amountIn;
#     uint256 amountOutMinimum;
#     uint160 sqrtPriceLimitX96;
# }

config.reload()
active_network = config.get_active_network()
uniswap_v3_swap_router = active_network.manifest_named("uniswap_v3_swap_router")
weth.approve(uniswap_v3_swap_router.address, weth_amount_to_sell_in_wei)

min_out = int(trades["usdc"]["trade"] * (10 ** 6) * 0.99)

print("Let's swap WETH for USDC")
uniswap_v3_swap_router.exactInputSingle(
    (weth.address, 
     usdc.address, 
     3000, 
     boa.env.eoa, 
     weth_amount_to_sell_in_wei, 
     min_out, 
     0
    )
)


print_token_balances()

Let's swap WETH for USDC
USDC balance: 951530154
WETH balance: 0
Aave USDC balance: 100000000
Aave WETH balance: 720489084145302785


In [72]:
print_token_balances()

USDC balance: 951530154
WETH balance: 0
Aave USDC balance: 100000000
Aave WETH balance: 720489084145302785


In [73]:
usdc.balanceOf(boa.env.eoa)

951530154

In [74]:
deposit(pool_contract, usdc, usdc.balanceOf(boa.env.eoa))
print_token_balances()

Depositing 951530154 USD Coin into Aave contract 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
USDC balance: 0
WETH balance: 0
Aave USDC balance: 1051530154
Aave WETH balance: 720489084145302785


In [85]:
print_token_balances()

USDC balance: 0
WETH balance: 0
Aave USDC balance: 1051530154
Aave WETH balance: 720489084145302785


In [87]:
a_usdc_balance = a_usdc.balanceOf(boa.env.eoa)
print(f"Aave USDC balance: {a_usdc_balance}")

a_weth_balance = a_weth.balanceOf(boa.env.eoa)
print(f"Aave WETH balance: {a_weth_balance}")

a_usdc_balance_normalized = a_usdc_balance / 10 ** 6
a_weth_balance_normalized = a_weth_balance / 10 ** 18

print(f"Aave USDC balance normalized: {a_usdc_balance_normalized}")
print(f"Aave WETH balance normalized: {a_weth_balance_normalized}")

usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * eth_price

total_value = usdc_value + weth_value

weth_percent_allocation = weth_value / total_value
usdc_percent_allocation = usdc_value / total_value

print(f"WETH percent allocation: {weth_percent_allocation:.2f}")
print(f"USDC percent allocation: {usdc_percent_allocation:.2f}")




Aave USDC balance: 1051530154
Aave WETH balance: 720489084145302785
Aave USDC balance normalized: 1051.530154
Aave WETH balance normalized: 0.7204890841453028
WETH percent allocation: 0.70
USDC percent allocation: 0.30
